---

# Data Exploration and Preparation

---

**Content**

- Data Exploration

- Data Preprocessing

- Feature Engineering

- Save Features to Disk



**Additional Material**

- pandas dashboard library [pandas-profiling](https://github.com/ydataai/pandas-profiling)

- geospatial data processing library  [cartopy](https://scitools.org.uk/cartopy/docs/latest/installing.html)


**Central Concepts**

- Data Leakage

**Instructions**

- Solve the tasks indicated by the keyword **TODO**

- Solve the tasks **sequentially**, variable names are recycled, can lead to inconsistencies.

- Read the description carefully (typo's are human, use common sense)

- Follow the structure given in the **TIP**

- **INSERT SOLUTION** indicates how many columns our solution had.



---
---

In [ ]:
# matplotlib color settings

dark_plot_theme = True

if dark_plot_theme:
    plt.style.use('dark_background')

In [ ]:
# load general dependencies
from collections import Counter
import datetime
import numpy as np
import os
import pandas as pd
from pathlib import Path
import re
import requests
import zipfile

# for geographic data
import cartopy.crs as ccrs
import geopandas as gpd
from geopy.distance import great_circle 

# visualization
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# load and refresh custom functions

import importlib
import utils
importlib.reload(utils)
from utils import get_dichotomous

In [ ]:
# pandas display settings

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)


In [ ]:
# prepare the working directory

cwd = Path()

opath = cwd / 'data'
ppath = cwd / 'plots'

opath.mkdir(exist_ok=True)
ppath.mkdir(exist_ok=True)

ifname = opath / 'listings.csv'
ofname = opath / 'features.csv'


# Load the Dataset

In [ ]:
# #extract specific columns of the raw data

index = ['id']

cnames = []
cnames += ["space"]
cnames += ["description"]
cnames += ["host_since"]
cnames += ["host_is_superhost"]
cnames += ["neighbourhood_group_cleansed"]
cnames += ["latitude"]
cnames += ["longitude"]
cnames += ["room_type"]
cnames += ["bathrooms"]
cnames += ["bedrooms"]
cnames += ["beds"]
cnames += ["amenities"]
cnames += ["square_feet"]
cnames += ["price"]
cnames += ["cleaning_fee"]
cnames += ["security_deposit"]
cnames += ["minimum_nights"]
cnames += ["number_of_reviews"]
cnames += ["review_scores_rating"]
cnames += ["review_scores_cleanliness"]
cnames += ["review_scores_location"]
cnames += ["instant_bookable"]
cnames += ["host_id"]
cnames += index

In [ ]:
# TODO:
# load in the csv file: "ifname"
# use pandas pd.read_csv
# and specify columns: "cnames"
# and the index column: "index"

# TIP:
# data = 

data = pd.read_csv(ifname, usecols=cnames, index_col=index).sort_index(axis=1) # REMOVE

# process and gather features in odata
odata = data.copy(deep=True)

In [ ]:
# CHECK

if len(data.columns) == 23:
    print('*** passed, well done!')
else:
    print('*** wrong number of columns, try again')

if data.index.name == 'id':
    print('*** passed, well done!')
else:
    print('*** wrong index, try again')

# Data Exploration

---

The first data exploration is done with\
the help of the pandas-profiling package.

The report is generated by running the notebook:

`create_repots.ipynb`

This will generated the report of the raw data:

./plots/data_report_raw.html

It is an html file so view it in the browser.

# Data Preprocessing

---

In [ ]:
# transform monetary string to numeric variables

variables = ['price', 'cleaning_fee', 'security_deposit']

result = pd.DataFrame()

for v in variables:
    # TODO:
    # The "variables" are strings convert them into floats.
    # Therefore, remove the characters: "$" and ",".
    # Use pandas "str.replace" and "astype".

    # TIP:
    # result[v] = data[v]...
    
    result[v] = data[v].str.replace('[$,]', '', regex=True).astype(float) # REMOVE

In [ ]:
# CHECK

passed = []

if result['price'].dtype == 'float':
    print('*** passed, well done!')
else:
    print('*** variables are not float, try again')
    passed.append(False)

if np.floor(result['price'].sum()) in [1514224, 1322728]:
    print('*** passed, well done!')
else:
    print('*** variables do not add up, try again')
    passed.append(False)

if False not in passed:
    odata[variables] = result[variables]



In [ ]:
# transform boolean strings to numbers

variables = ['host_is_superhost', 'instant_bookable']

d = {'t': 1., 'f': 0.}

result = pd.DataFrame()

for v in variables:
    # TODO:
    # Convert the "variables" from string to float.
    # Therefore, use the provided dictionary "d"
    # and the pandas ".map" function.
    # Furthermore, specify how to handle Null values in "map".

    # TIP:
    # result[v] = data[v]...
    
    result[v] = data[v].map(d, na_action='ignore') # REMOVE

In [ ]:
# CHECK

passed = []

if result['host_is_superhost'].sum() == 3011:
    print('*** passed, well done!')
else:
    print('*** variables do not add up, try again')
    passed.append(False)


if result['host_is_superhost'].isnull().sum() == 26:
    print('*** passed, well done!')
else:
    print('*** variables do not add up, try again')
    passed.append(False)

if False not in passed:
    odata[variables] = result[variables]

# Feature Engineering

---

In [ ]:
# extract specific amenities from the 

amenities = data['amenities'].str.strip('{}').str.replace('"', '').str.split(',')

In [ ]:
# TODO:
# Count the appearances of the individual amenities in the variables "amenities"
# First flatten the nested "amenities" variable with panda "explode" function.
# Then use the "Counter" class from the "collections" library.
# Counter: dict like, stores names as keys and counts as values

# TIP:
# counter = ...

counter = Counter(amenities.explode()) # REMOVE

In [ ]:
# CHECK

if counter['TV'] == 10134:
    print('*** passed, well done!')
else:
    print('*** the TV in counter does not add up, try again')

In [ ]:
# plot found amenities

# the 70 most frequent amenities
df_counter = pd.DataFrame(list(counter.items()), columns=['amenities', 'count']).sort_values('count', ascending=False).head(70)

fig, ax = plt.subplots(figsize=(8, 12))

# TODO:
# Visualize the amenities in the df_counter DataFrame.
# Use the "barplot" function from the "seaborn" (imported as sns) library
# Set the x to "count" and y to "amenities".
# What is the most frequent amenity?

# TIP:
# sns.barplot(...)

sns.barplot(data=df_counter, y='amenities', x='count'); # REMOVE

In [ ]:
# extract dummy variables for the amenities

features = []

nv = 'family'
pattern = 'Crib|Family/kid friendly|Baby|Children'
features += [(nv, pattern)]

nv = 'smoking'
pattern = 'Smoking allowed'
features += [(nv, pattern)]

nv = 'TV'
pattern = 'TV'
features += [(nv, pattern)]

nv = 'internet'
pattern = 'WiFi|Internet'
features += [(nv, pattern)]

nv = 'pets'
pattern = 'Pets allowed'
features += [(nv, pattern)]

nv = 'parking'
pattern = 'Parking|parking'
features += [(nv, pattern)]

v = 'amenities'

result = pd.DataFrame()

for nv, pattern in features:
    # TODO:
    # We create new dummy variables if special amenities are present.
    # The new variable names "nv" are looped over together with the "pattern".
    # If the pattern is present return true and cast to 1.
    # For this use pandas ".str.contains"
    # and set the type with "astype" to integer.

    # TIP:
    # result[nv] = data[v].str...
    
    result[nv] = data[v].str.contains(pattern).astype('uint8') # REMOVE

In [ ]:
# CHECK

passed = []

if result['internet'].sum() == 7909:
    print('*** passed, well done!')
else:
    print('*** the internet variable does not add up, try again')
    passed.append(False)

if False not in passed:
    odata = pd.concat([odata, result], axis=1)

In [ ]:
# sum up kitchen amenities

nv = 'basic_kitchen'
dtype = 'uint8'
odata[nv] = np.sum([odata[v].str.contains('Coffee').astype(dtype),
                  odata[v].str.contains('Dishes').astype(dtype),
                  odata[v].str.contains('Oven').astype(dtype),
                  odata[v].str.contains('Dishwasher').astype(dtype),
                  odata[v].str.contains('Microwave').astype(dtype),
                  odata[v].str.contains('Refrigerator').astype(dtype),
                  odata[v].str.contains('Dishwasher').astype(dtype),
                  odata[v].str.contains('Cooking basics').astype(dtype)],
                 axis=0)

In [ ]:
# extract square meters from the description

v = 'square_feet'
nv = 'square_meter'

pattern = '(\d{2,4})\s?(sq\s*m|square\s*m|quadrate\s*meter|m2|m\^2|m²|mq)'
m_per_ft = 0.3048

sqm_from_desc = data['description'].str.extract(pattern, flags=re.IGNORECASE)[0].astype('float64')

# TODO:
# Combine the data from the dataframe column "square_feet"
# with the values extracted into "sqm_from_desc"
# First change units from SQUARE feet to SQUARE meter using m_per_ft**2
# Then use the pandas function "combine_first"
# To combine the output in to the new variable "square_meter" (nv)

# TIP:
# odata[nv] = (data[v] * ... ).combine_first( ... )

odata[nv] = (data[v] * m_per_ft**2).combine_first(sqm_from_desc) # REMOVE


In [ ]:
# CHECK

if np.floor(odata['square_meter'].sum()) == 285489:
    print('*** passed, well done!')
else:
    print('*** the square_meter variable does not add up, try again')

In [ ]:
# TODO:
# Replace all zeros with np.nan
# in the "square_meter" variable.
# Use the pandas "replace" function

# TIP:
# result = odata[nv]...

result = odata[nv].replace(0, np.nan) # REMOVE

In [ ]:
# CHECK

passed = []

if (result < 0.1).sum() == 0:
    print('*** passed, well done!')
else:
    print('*** the numbers do not add up, try again')
    passed.append(False)

if False not in passed:
    odata[nv] = result

In [ ]:
# count the number of listings per host
# using a pandas cookbook trick

# TODO:
# Create the new variable 'listings/host' in :
# which contains the count of listings per host
# This trick works by grouping the data after the "host_id"
# and then using "transform" to count the elements in each group
# with the "len" function
# But you need a second dummy variable: "beds"

# TIP:
# result = odata[['host_id', ...]].groupby(...).transform(...);

result = odata[['host_id', 'beds']].groupby("host_id").transform(len); # REMOVE

In [ ]:
# CHECK

passed =[]

if result.loc[2015].values[0] == 4:
    print('*** passed, well done!')
else:
    print('*** the variable does not add up, try again')
    passed.append(False)

if False not in passed:
    odata['listings/host'] = result

# Binning

In [ ]:
# extraction of years_registered

class YSR:
    def __init__(self, dates):
        self.reference = dates.max()

    def __call__(self, date):
        return np.abs((self.reference - date).days / 356)

# TODO:
# To calculate the "years_registered" feature
# The feature "host_since" needs to be converted to_datetime.
# Use the pandas utility function to_datetime

# TIP:
# host_since = pd...

host_since = pd.to_datetime(odata['host_since']) # REMOVE


In [ ]:
# CHECK

passed = []

if isinstance(host_since.iloc[0], datetime.datetime):
    print('*** passed, well done!')
else:
    print('*** the variable does not add up, try again')
    passed.append(False)

if False not in passed:
    odata['years_registered'] = host_since.apply(YSR(host_since))

In [ ]:
# binning of years_registered

bins = [0, 1, 4, np.inf]

groups = ['Newbies','Experienced','Professionals']

# TODO:
# Use pandas cut function to bin "years_registered"
# into "bins" with the labels given by "groups".
# Save the result in the feature "host_since_cat".

# TIP:
# result = pd.cut(...)

result = pd.cut(odata['years_registered'], bins, labels=groups) # REMOVE

In [ ]:
# CHECK

passed = []

if 'Professionals' in result.unique():
    print('*** passed, well done!')
else:
    print('*** no Professionals found, try again')
    passed.append(False)

if False not in passed:
    odata['host_since_cat'] = result

In [ ]:
# show years_registered

# TODO:
# Take a quick look at the results.
# Use the seaborn "countplot" routine.

# TIP:
# sns.countplot(...)

sns.countplot(x=odata['host_since_cat']); # REMOVE

## Dummy Encode

In [ ]:
# dummy encode

vs = ['room_type', 'neighbourhood_group_cleansed', 'host_since_cat']

# TODO:
# Create dummy variable from the variables in "vs"
# Use the dedicated pandas convenience function
# for this task: "get_dummies"

# TIP:
# dummies =

dummies = pd.get_dummies(odata[vs]) # REMOVE

In [ ]:
# CHECK

passed = []

if len(dummies.columns) == 18:
    print('*** passed, well done!')
else:
    print('*** wrong number of dummy variables found, try again')
    passed.append(False)

if False not in passed:
    odata = odata.join(dummies)

## Features from Location

In [ ]:
# download/extract berlin shape files

url = "https://tsb-opendata.s3.eu-central-1.amazonaws.com/bezirksgrenzen/bezirksgrenzen.shp.zip"

shp_folder = opath / 'berlin_shape_files'
shp_folder.mkdir(exist_ok=True)
shp_fname_zip = shp_folder / os.path.basename(url)

response = requests.get(url)
open(shp_fname_zip, "wb").write(response.content);

with zipfile.ZipFile(shp_fname_zip, 'r') as zip_ref:
    zip_ref.extractall(shp_folder)

shp_fname = list(opath.rglob('*.shp'))[0]

In [ ]:
# plot spatial price distribution

plt.rcParams['axes.grid'] = False

figsize = (12, 8)
fig = plt.figure(figsize=figsize)
proj = ccrs.PlateCarree()
ax = fig.add_subplot(111, projection=proj)

berlin = gpd.read_file(shp_fname)
berlin.plot(edgecolor='black', ax=ax);

scatter_plot = ax.scatter(odata['longitude'], odata['latitude'], c=odata['price'], cmap='inferno_r', vmax=600, s=0.7)
cbar = plt.colorbar(scatter_plot, ax=ax)
cbar.ax.set_ylabel('price')

ax.set_title('Berlin', fontsize='xx-large')

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0)
gl.top_labels = False
gl.right_labels = False

plt.show()

Add variables that contain the distance to important places like the central station.

In [ ]:
# calculate distances to special places


# dict of special places: coordinates (lat, lon)
locations = {}
locations['Zoo'] = (52.507216, 13.332271)
locations['FreieUni'] = (52.452526, 13.289679)
locations['Potsdamer'] = (52.508969, 13.376300)
locations['Kottbusser'] = (52.499083, 13.418140)
locations['Rosenthaler'] = (52.529650, 13.401321)
locations['Hauptbahnhof'] = (52.52493, 13.369181)

# TODO:
# Add the coordinates of an interesting place to the dictionary
# TIP:
# locations['...'] = (...)

# calculate distances to all rows
for k, place in locations.items():
    distances = lambda x: great_circle((x['latitude'], x['longitude']), place).km

    if 'dist_{k}' not in odata:
        odata[f'dist_{k}'] = odata.apply(distances, axis=1)


# Impute Outliers

- calculate the z-scores
- here, outliers have a Z-Score > 3
- visualize the data, with and withodata outliers and log transformation

In [ ]:
# class to calculate z-scores

class ZScore:
    def __init__(self, d):
        self.m = np.mean(d)
        self.s = np.std(d)

    def __call__(self, x):
        return np.abs((x - self.m) / self.s)

In [ ]:
# impute z-scores > 3 with the mean

# exclude specific variables
exclude = []
exclude += ['dist']
exclude += ['lat']
exclude += ['lon']
exclude += ['host_id']
exclude += ['square_feet']
exclude += get_dichotomous(odata)

pattern = f'^(?!{"|".join(exclude)}).*'
variables = odata.select_dtypes(np.number).filter(regex=pattern, axis=1).columns

# mean impute z-scores > 3
for v in variables:
    # TODO:
    # Create the outlier mask
    # which is true for z-scores > 3.
    # Instantiate the "ZScore" class
    # and apply it like a function.

    # TIP:
    # mask = odata[v].apply(...) > ...

    mask = odata[v].apply(ZScore(odata[v])) > 3 # REMOVE

    contains_outliers = mask.sum() > 0
    if contains_outliers:
        odata[f'imp_z_{v}'] = mask.astype(int)
        odata.loc[mask, v] = odata[v].mean()


In [ ]:
# CHECK

if len(odata.filter(regex='^imp_z.*').columns) == 14:
    print('*** passed, well done!')
else:
    print('*** wrong number of variables found, try again')

In [ ]:
# plots for neighbourhood price

figsize = (12, 15)

fig, axs = plt.subplots(3, figsize=figsize, sharex=True)

vs = ['neighbourhood_group_cleansed', 'price']
tmp = odata[vs]

# boxplot
sns.boxplot(x=vs[0], y=vs[1], data=odata, showfliers=False, ax=axs[0])

# TODO:
# Create a violin plot of the "price" for each "neighbourhood_group_cleansed".
# Use the example below as a template.
# Compare the results with and without log transformation.

# TIP:
# sns.violinplot(...)

sns.violinplot(data=odata, x=vs[0], y=vs[1], ax=axs[1]) # REMOVE


# violin plot with log price

tmp['log_price'] = np.log(tmp['price']+1)
sns.violinplot(data=tmp, x=vs[0], y='log_price', ax=axs[2])

axs[-1].tick_params(axis='x', rotation=25)


# Impute Missing Values

- with imputation indicator variables

In [ ]:
# impute with "0"

vs = ["cleaning_fee", "security_deposit", 'host_is_superhost', 'years_registered']
nvs = [f'imp_{v}' for v in vs]

if nvs[0] not in odata:
    # TODO:
    # Create the variables from the "nvs" list,
    # which hold the 0/1 indicator if a variable is imputed.
    # Imputed -> True -> 1
    # Use the pandas functions "isnull" and "astype".

    # TIP:
    # odata[nvs] = odata[vs]...

    odata[nvs] = odata[vs].isnull().astype(int) # REMOVE

    # TODO:
    # Fill null values with 0
    # for the variables in "vs"
    # Use pandas "fillna" function.
    
    # TIP:
    # result = ...
    
    result = odata[vs].fillna(0) # REMOVE

In [ ]:
# CHECK

passed = []

if odata['imp_security_deposit'].sum() == 9361:
    print('*** passed, well done!')
else:
    print('*** wrong number of imputations found, try again')
    passed.append(False)

if (result['cleaning_fee'] == 0).sum() == 9011:
    print('*** passed, well done!')
else:
    print('*** wrong number of zeros found, try again')
    passed.append(False)

if False not in passed:
    odata[vs] = result

In [ ]:
# impute with the mean

vs = []
vs += ['review_scores_rating']
vs += ['review_scores_cleanliness']
vs += ['review_scores_location']
vs += ['bathrooms']
vs += ['bedrooms']
vs += ['beds']
vs += ['square_meter']

nvs = [f'imp_{v}' for v in vs]

if nvs[0] not in odata:
    odata[nvs] = odata[vs].isnull().astype(int)

    # TODO:
    # Impute the values with their respective mean value
    # Use pandas "fillna" function

    # TIP:
    # result = odata[vs]...
    
    result = odata[vs].fillna(odata[vs].mean()) # REMOVE

In [ ]:
# CHECK

passed = []

if odata['imp_review_scores_cleanliness'].sum() == 4411:
    print('*** passed, well done!')
else:
    print('*** wrong number of imputations found, try again')
    passed.append(False)

if result['review_scores_cleanliness'].isna().sum() == 0:
    print('*** passed, well done!')
else:
    print('*** still na values found, try again')
    passed.append(False)

if False not in passed:
    odata[vs] = result

In [ ]:
# impute with the mode

vs = []
vs += ['host_since']
vs += ['host_since_cat']

nvs = [f'imp_{v}' for v in vs]

result = pd.DataFrame()

for v, nv in zip(vs, nvs):
    if nv not in odata:
        odata[nv] = odata[v].isnull().astype(int)

    if v in odata:

        # TODO:
        # Impute the values with their respective mode value
        # Use pandas "fillna" function
        # Be careful with the return type of mode, may need some indexing

        # TIP:
        # result[v] = odata[v]...
        
        result[v] = odata[v].fillna(odata[v].mode()[0]) # REMOVE


In [ ]:
# CHECK

passed = []

if odata['imp_host_since'].sum() == 26:
    print('*** passed, well done!')
else:
    print('*** wrong number of imputations found, try again')
    passed.append(False)

if 'host_since' in odata:
    if result['host_since'].isna().sum() == 0:
        print('*** passed, well done!')
    else:
        print('*** still na values found, try again')
        passed.append(False)

    if False not in passed:
        odata[vs] = result[vs]

# Drop Variables

In [ ]:
vs = []
vs += ['description']
vs += ['space']
vs += ['square_feet']
vs += ['amenities']
vs += ['host_since']

# TODO:
# Why would including
# "imp_z_price" be data leakage?

vs += ['imp_z_price']

odata = odata.drop(vs, axis=1, errors='ignore')

In [ ]:
# check for nullity

tmp = odata.isnull().sum() / len(data) * 100
for k, v in tmp.items():
    if v > 0:
        print(f'{k} {v:.3f}')

In [ ]:
# check for variables without variance

for k, v in odata.items():
    u = pd.unique(v)
    if len(v) < 2:
        print(k)

# Write Data to Disk


---

In [ ]:
# Well done! Now the results.

odata.to_csv(ofname, header = True, index = True)

# Feature Data Report

To check your features\
please generate the feature report\
by running the notebook:

`create_repots.ipynb`

This will generated the report of the feature data:

./plots/data_report_features.html

It is an html file so view it in the browser.

In [ ]:
# TODO:
# there could be more features to be discovered and extracted.
# Have a look at the categorical variables.

variables = ['description', 'space']
data[variables]

---
---
---